<a href="https://www.kaggle.com/code/gabriellagloria/uni-prediction-using-neural-network?scriptVersionId=142969330" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read data
uni = pd.read_csv("/kaggle/input/data-for-admission-in-the-university/adm_data.csv")
uni2 = uni
uni.head()

# Explore Data

In [ ]:
uni.info()

In [ ]:
uni = uni.set_index('Serial No.')
uni2 = uni2.set_index('Serial No.')

In [ ]:
uni["Chance of Admit "].describe()

In [ ]:
uni.corr()

In [ ]:
# 367/400 people has >= 0.5 chance 
uni2["Chance of Admit "].sum()
uni2["Chance of Admit "].describe()

In [ ]:
# Round chance of admit to true (chance >= 0.5) or false (chance < 0.5)
uni2['Chance of Admit '] = (uni2['Chance of Admit '] >= 0.5)
# Correlation gets worse if we round the chance
uni2.corr()

In [ ]:
# Correlation heatmap
import seaborn as sns
uni.corr()
sns.heatmap(uni.corr())

CGPA has biggest correlation with chance of admit compared to other features

In [ ]:
import plotly.express as px
fig = px.scatter(uni, x ='CGPA', y = 'Chance of Admit ' , title = 'CGPA vs Chance of Admit')
fig.show()

In [ ]:
fig = px.scatter(uni2, x ='CGPA', y = 'Chance of Admit ' , title = 'CGPA vs Chance of Admit')
fig.show()

In [ ]:
uni[["Chance of Admit ","CGPA"]].hist(bins = 10)

In [ ]:
uni.hist(bins = 20)

In [ ]:
plt.bar(x = uni["CGPA"], height = uni["Chance of Admit "])
plt.xlabel("CGPA")
plt.ylabel("Chance of Admit ")
plt.title("CGPA vs Chance of Admit")

# Split Data

In [ ]:
from sklearn.model_selection import train_test_split

X = uni.drop(["Chance of Admit "], axis = 1)
y = uni["Chance of Admit "]

# Test size 0.2, 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
from sklearn.metrics import mean_absolute_error
# Make baseline from "Chance of Admit " mean  
mean_predict_train = [y_train.mean()] * len(y_train)
print("Train MAE = ", mean_absolute_error(y_train, mean_predict_train))

mean_predict_test = [y_test.mean()] * len(y_test)
print("Test MAE = ", mean_absolute_error(y_test, mean_predict_test))

# Build Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(16, activation='relu', input_shape=[X_train.shape[1]]),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    #layers.Dense(8, activation='relu'), 
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
# add optimizer, loss, and metric
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['accuracy'],
)


In [ ]:
# doesn't overfit , no early stopping needed
'''
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)
'''

history = model.fit(
    X_train, y_train,
    batch_size=16,
    epochs=500,
    verbose=0,
    #callbacks = [early_stopping]
)

In [ ]:
pred = model.predict(X_test)

In [ ]:
model_mae = mean_absolute_error(pred, y_test)

# Trained Model MAE vs Mean Baseline MAE
print("Model MAE = ", model_mae)
print("Baseline MAE = ", mean_absolute_error(y_test, mean_predict_test))

In [ ]:
sns.scatterplot(x=y_test, y=pred.flatten());
plt.xlabel('y_test')
plt.ylabel('Our predictions')
plt.title('Test Data vs Predicted Data ')
plt.show()

Thank you :D !